<a href="https://colab.research.google.com/github/Amelrich/Capstone-Fall-2020/blob/master/padded%26skipped-dct-pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import os
current = os.getcwd()
print(current)
os.chdir('../utilities')
print(os.getcwd())

C:\Users\Max Tchibozo\Desktop\CAPSTONE\Capstone-Fall-2020\pipelines
C:\Users\Max Tchibozo\Desktop\CAPSTONE\Capstone-Fall-2020\utilities


In [4]:
os.chdir(current)

In [14]:
import numpy as np
import pandas as pd
#import pandas_datareader.data as web
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing  import MinMaxScaler

import sys
sys.path.insert(1, '../utilities/')
from TS_utils import DCT_lowpass_filter, TS_generator

#import seaborn as sns


In [15]:
TS_generator()

HTTPError: HTTP Error 404: Not Found

In [6]:
!pip install tslearn

     |████████████████████████████████| 778kB 6.0MB/s 


In [7]:
from tslearn.clustering import TimeSeriesKMeans

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.cluster.k_means_ module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.cluster. Anything that cannot be imported from sklearn.cluster is now part of the private API.
  warnings.warn(message, FutureWarning)


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Important Note

Make sure you have Maxime's TS_utils.py in in this directory so you can import DCT smoothing

In [9]:
X = None # YOUR TIMESERIES GO HERE
#X = np.load("/content/drive/My Drive/capstone/timeseries.npy")

In [13]:
################# Helping functions ################# 

def get_multiscale_skipped_values(df_start_end):
  days_60 = df_start_end[-60:]
  days_40 = [0 if i % 2 else x for i, x in enumerate(days_60)]
  days_20 = [0 if i % 3 else x for i, x in enumerate(days_60)]
  dictionary = {'60_days': np.array(days_60), '40_days': np.array(days_40), 
                '20_days': np.array(days_20)}
  return dictionary

def get_multiscale_consecutive_values(df_start_end):
  days_20 = df_start_end[-20:]
  days_40 = df_start_end[-40:]
  days_60 = df_start_end[-60:]
  
  days_20_all = np.append(np.zeros((40)), days_20 ,0)
  days_40_all = np.append(np.zeros((20)), days_40,0)
  
  dictionary = {'20_days': np.array(days_20_all), '40_days': np.array(days_40_all),
                    '60_days': days_60}
  return dictionary

#################################################################### 

def preprocessing(X):
  print("Preprocessing Start")
  # scale
  X_Scaled = []
  for df in X:
    scaler = MinMaxScaler()
    X_Scaled.append(scaler.fit_transform(df[:,0].reshape(-1, 1)).reshape(1,-1)[0])  

  # dct smooth
  lowpass_filter = DCT_lowpass_filter()
  X_dct_reconstructed = lowpass_filter.fit_transform(np.array(X_Scaled))
  X_dct = lowpass_filter.X_dct
  n_dct = lowpass_filter.nb_coefs

  # create the 3d arrrays for both methods
  skipped_values = []
  consecutive_values = []
  n = 0
  for df in X_dct_reconstructed:
    n = n + 1
    if n%1000 == 0:
      print("{} stocksout of {} completed".format(n,len(X_dct_reconstructed)))
  
    dictionary_skipped_values = get_multiscale_skipped_values(df)
    dictionary_consecutive_values = get_multiscale_consecutive_values(df)
 
    datapoint_skipped_values = pd.DataFrame(dictionary_skipped_values).to_numpy()
    datapoint_consecutive_values = pd.DataFrame(dictionary_consecutive_values).to_numpy()
    skipped_values.append(datapoint_skipped_values)
    consecutive_values.append(datapoint_consecutive_values)
  
  print("Preprocessing Done")
  return skipped_values, consecutive_values

def clustering(x):
  num_clus = 6
  km_skipped = TimeSeriesKMeans(n_clusters=num_clus, max_iter=5, metric='dtw',random_state=0).fit(x)
  return km_skipped.predict(x)

def preprocessing_clustering(x):
  skipped_values, consecutive_values  = preprocessing(x)
  print("Starting to cluster both arrays...")
  x_clusters_skipped = clustering(skipped_values)
  x_clusters_consecutive = clustering(consecutive_values)
  print("Finished!")
  return x_clusters_skipped, x_clusters_consecutive

In [14]:
x_clusters_skipped, x_clusters_consecutive = preprocessing_clustering(X)

Preprocessing Start
1000 stocksout of 4000 completed
2000 stocksout of 4000 completed
3000 stocksout of 4000 completed
4000 stocksout of 4000 completed
Preprocessing Done
Starting to cluster both arrays...
Finished!


In [15]:
x_clusters_skipped

array([4, 3, 0, ..., 3, 1, 2])

In [ ]:
x_clusters_consecutive